# Importing Libraries

In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
try:
    import dill as pickle
except ImportError:
    import pickle

# Loading Documents

In [4]:
# load doc into memory
def load_doc(filename): 
    file = open(filename, 'r', encoding='utf8') 
    text = file.read() 
    file.close()
    return text

#filename = "./drive/My Drive/Flickr8K_Text/Flickr8k.token.txt"
#filename = "./drive/My Drive/Project/Theme_data_text/themedata.token.txt"
filename = "./drive/My Drive/Project/urdu/urdu_token.txt"
# load descriptions
doc = load_doc(filename)
print(doc[:300])

﻿1015118661_980735411b.jpg#0	لڑکا شہر میں پتھریلی دیوار کے سامنے مسکرا رہا ہے۔
1015118661_980735411b.jpg#1	چھوٹا لڑکا سڑک پر کھڑا ہے جبکہ آدمی پتھر کی دیوار پر کام کر رہا ہے
1015118661_980735411b.jpg#2	لڑکا گلی میں بھاگتے ہوئے
1015118661_980735411b.jpg#3	بچہ سڑک پہ چل رہا ہے اور اس کے پیچھے آدمی ہے



In [5]:
def load_descriptions(doc):
    mapping = dict()
    
    for line in doc.split('\n'):
        
        if len(line) < 2:
            continue
        tokens = line.split()
        
        image_id, image_desc = tokens[0], tokens[1:]
        
        image_id = image_id.split('.')[0]
        
        image_desc = ' '.join(image_desc)
        
        if image_id not in mapping:
            mapping[image_id] = list()
        
        mapping[image_id].append(image_desc)
    return mapping

descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Loaded: 552 


In [6]:
list(descriptions.items())[:1]

[('\ufeff1015118661_980735411b',
  ['لڑکا شہر میں پتھریلی دیوار کے سامنے مسکرا رہا ہے۔'])]

In [7]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def clean_descriptions(descriptions):
    
    table = str.maketrans('', '', string.punctuation)
    for key, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            
            desc = tokenizer.tokenize(desc)
            
            desc = [word.lower() for word in desc]
            
            desc = [word for word in desc if len(word)>1]
            
            desc_list[i] =  ' '.join(desc)

clean_descriptions(descriptions)

In [8]:
descriptions['1015118661_980735411b']

['چھوٹا لڑکا سڑک پر کھڑا ہے جبکہ آدمی پتھر کی دیوار پر کام کر رہا ہے',
 'لڑکا گلی میں بھاگتے ہوئے',
 'بچہ سڑک پہ چل رہا ہے اور اس کے پیچھے آدمی ہے',
 'سفید قمیض اور نیلی پتلون میں مسکراتا بچہ جس کے پیچھے آدمی ہے']

In [9]:
# convert the loaded descriptions into a vocabulary of words
def getVocabulary(descriptions):
    # build a list of all description strings
    all_desc = set()
    for key in descriptions.keys():
        [all_desc.update(tokenizer.tokenize(d)) for d in descriptions[key]]
    return all_desc

# summarize vocabulary
vocabulary = getVocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 1561


In [10]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

#save_descriptions(descriptions, 'descriptions.txt')
save_descriptions(descriptions, 'urdu_descriptions.txt')
print("Desciption.txt created")

Desciption.txt created


# Preparing Data for training

In [11]:
# load Train Data
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    
    for line in doc.split('\n'):
        
        if len(line) < 1:
            continue
 
        dataset.append(line)
    return (dataset)
 
#filename = './drive/My Drive/Flickr8K_Text/Flickr_8k.trainImages.txt'
#filename = './drive/My Drive/Project/Theme_data_text/train.names.txt'
filename = './drive/My Drive/Project/urdu/train_urdu.txt'
train_names = load_set(filename)
print('Train size: %d' % len(train_names))

#Directory_path = './drive/My Drive/Flickr8K_Data/'
Directory_path = './drive/My Drive/Project/Theme_dataset/'
train_img= [Directory_path+i  for i in train_names]

Train size: 500


In [12]:
# Loading test data, 
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    
    for line in doc.split('\n'):
        
        if len(line) < 1:
            continue
 
        dataset.append(line)
    return (dataset)
 
#filename = './drive/My Drive/Flickr8K_Text/Flickr_8k.testImages.txt'
#filename = './drive/My Drive/Project/Theme_data_text/test.names.txt'
filename = './drive/My Drive/Project/urdu/test_urdu.txt'
test_names = load_set(filename)
print('Test Size: %d' % len(test_names))

#Directory_path = './drive/My Drive/Flickr8K_Data/'
Directory_path = './drive/My Drive/Project/Theme_dataset/'
test_img= [Directory_path+i  for i in test_names]

Test Size: 50


In [13]:
# we r extracting train data captions such that, each captions starts with 'startseq' and ends with 'endseq
def load_clean_descriptions(filename, dataset):
    # load document
    dataset = [i.split('.')[0] for i in dataset]
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        
        tokens = line.split()
        
        image_id, image_desc = tokens[0], tokens[1:]
        
        if image_id in dataset:
            
            if image_id not in descriptions:
                descriptions[image_id] = list()
            
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            
            descriptions[image_id].append(desc)
    return descriptions

# descriptions
train_descriptions = load_clean_descriptions('urdu_descriptions.txt', train_names)

print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=500


In [14]:
# Save train_descriptions
#with open("train_descriptions.pkl", "wb") as encoded_pickle:
    #pickle.dump(train_descriptions, encoded_pickle)
#with open("theme_train_descriptions.pkl", "wb") as encoded_pickle:
with open("urdu_train_descriptions.pkl", "wb") as encoded_pickle:
    pickle.dump(train_descriptions, encoded_pickle)

# Extracting feature Vectors from Images

In [15]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    
    x = preprocess_input(x)
    return x

In [16]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

96116736/96112376 [==============================] - 9s 0us/step


In [17]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [18]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [19]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) 
    v = model_new.predict(image) 
    v = np.reshape(v, v.shape[1]) 
    return v

In [20]:
# Encoding all the Training Dataset to a 2048 Dimension Vectors, Run this once
start = time()
k=0
encoded_train = {}
for img in tqdm(train_img):
 
    encoded_train[img[len(Directory_path):]] = encode(img)
print("Time taken in seconds =", time()-start)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



Time taken in seconds = 592.4767067432404


In [21]:
# Encoding all the Test Dataset to a 2048 Dimension Vectors, Run this once
start = time()
encoded_test = {}
for img in tqdm(test_img):
    encoded_test[img[len(Directory_path):]] = encode(img)
print("Time taken in seconds =", time()-start)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



Time taken in seconds = 37.16768765449524


In [22]:
# Save Train features
#with open("encoded_train_images.pkl", "wb") as encoded_pickle:
    #pickle.dump(encoded_train, encoded_pickle)
#with open("theme_encoded_train_images.pkl", "wb") as encoded_pickle:
with open("urdu_encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoded_train, encoded_pickle)

In [23]:
# Save test features 

from pickle import dump, load
#with open("encoded_test_images.pkl", "wb") as encoded_pickle:
 #   pickle.dump(encoded_test, encoded_pickle)

#with open("theme_encoded_test_images.pkl", "wb") as encoded_pickle:
with open("urdu_encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoded_test, encoded_pickle)

 #   End Of this notebook

In [24]:
for key, c in encoded_train.items():
    print(key)

1015118661_980735411b.jpg
103106960_e8a41d64f8.jpg
1034276567_49bb87c51c.jpg
1042020065_fb3d3ba5ba.jpg
1052358063_eae6744153.jpg
1057089366_ca83da0877.jpg
106490881_5a2dd9b7bd.jpg
1075881101_d55c46bece.jpg
1077546505_a4f6c4daa9.jpg
1079274291_9aaf896cc1.jpg
1087168168_70280d024a.jpg
1095476286_87d4f8664e.jpg
1096165011_cc5eb16aa6.jpg
1096395242_fc69f0ae5a.jpg
109671650_f7bbc297fa.jpg
1100214449_d10861e633.jpg
1104133405_c04a00707f.jpg
1131340021_83f46b150a.jpg
1131932671_c8d17751b3.jpg
114051287_dd85625a04.jpg
114949897_490ca7eaec.jpg
118309463_a532b75be9.jpg
1204996216_71d7519d9a.jpg
1207159468_425b902bfb.jpg
1222322358_225067636e.jpg
1224851143_33bcdd299c.jpg
1236951314_0308dc4138.jpg
1236964638_1808784a3c.jpg
1244485675_822e6efe60.jpg
1253275679_e955fb7304.jpg
1255504166_f2437febcb.jpg
1269470943_ba7fc49b4d.jpg
1277743944_f4e8c78403.jpg
1285067106_2adc307240.jpg
1287475186_2dee85f1a5.jpg
1289142574_2bd6a082dd.jpg
1298295313_db1f4c6522.jpg
1299459550_1fd5594fa2.jpg
1301140633_046e4e8

In [25]:
train_img[0][len(Directory_path):]

'1015118661_980735411b.jpg'

In [26]:
len(encoded_train)

500